# Output-Parser

Wenn wir KI/Sprachmodelle in unsere Anwendung integrieren, dann ist es sicherlich sehr nützlich, wenn wir die Antworten der KI programmatisch verarbeiten können.
Da die KI immer nur Text zurückliefert, müssen wir also die Antwort parsen, z.B. um eine Dictionary zu erstellen.
Es gibt viele Anwendungsfälle, z.B. könnten wir ein Programm schreiben, das die Aufgabe hat Bewertungskommentare auszuwerten. Also sowas wie kurze Zusammenfassung, positiv/negativ, Sprache usw.
Schauen wir uns erstmal das Ganze ohne das Parsen an

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

bewertungen = [
    "Das ist wirklich ein tolles Produkt! Ich würde es auf jeden Fall nochmal bestellen. Besonders die Bildqualität finde ich beeindruckend!",
    "I bought this product last month. I haven't tested it until now. I will do that during the next few weeks.",
    "Ich weiß nicht was ich davon halten soll. Das Bild ist nicht scharf und die Farben sind blass. Ich würde es eher nicht empfehlen!"
]

prompt = (
    PromptTemplate.from_template(
        """Bitte analysiere folgende Bewertungskommentare. Erstelle für jeden Kommentar eine Zusammenfassung als einen Satz.
        Analysiere ausserdem ob die Kommentare positiv, neutral oder negativ sind.
        Gebe ausserdem die Sprache des Kommentars aus.

        Die Kommentare:
        {comments}
        """)
)

chain = LLMChain(llm=ChatOpenAI(), prompt=prompt)

response = chain.run(comments=bewertungen)

print(response)

Zusammenfassungen:
1. Das Produkt wird als toll und beeindruckend mit guter Bildqualität bewertet und der Kunde würde es wieder bestellen.
2. Der Kunde hat das Produkt letzten Monat gekauft, aber noch nicht getestet und plant dies in den nächsten Wochen zu tun.
3. Der Kunde ist unsicher und findet das Bild unscharf und die Farben blass. Er würde das Produkt nicht empfehlen.

Bewertungen:
1. Positiv
2. Neutral
3. Negativ

Sprache:
1. Deutsch
2. Englisch
3. Deutsch


Die KI hat die Aufgabe korrekt ausgeführt. Allerdings ist sie nicht optimal, wenn wir das z.B. als Datenstruktur in unserem Programm haben möchten, z.B. um es in einer Datenbank zu speichern oder um die Daten irgendwie in einer Oberfläche darzustellen.

Wir können aber das Format auch vorgeben. Machen wir das erstmal ohne die _Magie_ von LangChain.

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

bewertungen = [
    "Das ist wirklich ein tolles Produkt! Ich würde es auf jeden Fall nochmal bestellen. Besonders die Bildqualität finde ich beeindruckend!",
    "I bought this product last month. I haven't tested it until now. I will do that during the next few weeks.",
    "Ich weiß nicht was ich davon halten soll. Das Bild ist nicht scharf und die Farben sind blass. Ich würde es eher nicht empfehlen!"
]

prompt = (
    PromptTemplate.from_template(
        """Bitte analysiere folgende Bewertungskommentare. Erstelle für jeden Kommentar eine Zusammenfassung als einen Satz.
        Analysiere ausserdem ob die Kommentare positiv, neutral oder negativ sind.
        Gebe ausserdem die Sprache des Kommentars aus.

        Die Auswertung soll als JSON-Format ausgegeben werden. Felder: summary, sentiment, language

        Die Kommentare:
        {comments}
        """)
)

chain = LLMChain(llm=ChatOpenAI(), prompt=prompt)

response = chain.run(comments=bewertungen)

print(response)

{
  "comments": [
    {
      "summary": "Das ist wirklich ein tolles Produkt! Ich würde es auf jeden Fall nochmal bestellen. Besonders die Bildqualität finde ich beeindruckend!",
      "sentiment": "positiv",
      "language": "Deutsch"
    },
    {
      "summary": "I bought this product last month. I haven't tested it until now. I will do that during the next few weeks.",
      "sentiment": "neutral",
      "language": "Englisch"
    },
    {
      "summary": "Ich weiß nicht was ich davon halten soll. Das Bild ist nicht scharf und die Farben sind blass. Ich würde es eher nicht empfehlen!",
      "sentiment": "negativ",
      "language": "Deutsch"
    }
  ]
}


Das ist schon wesentlich besser. LangChain hilft uns bei derartigen Aufgaben. Lasst uns schauen, wie man es mit LangChain macht

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

response_schemas = [
    ResponseSchema(name="summary", description="Die Zusammenfassung des Kommentars als ein Satz"),
    ResponseSchema(name="sentiment", description="Ist der Kommentar positiv, negativ oder neutral"),
    ResponseSchema(name="language", description="In welcher Sprache ist der Kommentar verfasst")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

bewertungen = [
    "Das ist wirklich ein tolles Produkt! Ich würde es auf jeden Fall nochmal bestellen. Besonders die Bildqualität finde ich beeindruckend!",
    "I bought this product last month. I haven't tested it until now. I will do that during the next few weeks.",
    "Ich weiß nicht was ich davon halten soll. Das Bild ist nicht scharf und die Farben sind blass. Ich würde es eher nicht empfehlen!"
]

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="Bitte analysiere folgenden Bewertungskommentar.\n{format_instructions}\n{comment}",
    input_variables=["comment"],
    partial_variables={"format_instructions": format_instructions}
)

chain = LLMChain(llm=ChatOpenAI(), prompt=prompt)

for kommentar in bewertungen:
    response = chain.run(comment=kommentar)
    analysis = output_parser.parse(response)
    


{'summary': 'Das ist wirklich ein tolles Produkt!', 'sentiment': 'positiv', 'language': 'Deutsch'}
{'summary': "I bought this product last month. I haven't tested it until now. I will do that during the next few weeks.", 'sentiment': 'neutral', 'language': 'English'}
{'summary': 'Ich weiß nicht was ich davon halten soll. Das Bild ist nicht scharf und die Farben sind blass. Ich würde es eher nicht empfehlen!', 'sentiment': 'negativ', 'language': 'de'}


In [5]:
print(analysis)
output_parser.parse(analysis)

```json
{
	"summary": "Das ist wirklich ein tolles Produkt! Ich würde es auf jeden Fall nochmal bestellen. Besonders die Bildqualität finde ich beeindruckend!",
	"sentiment": "positiv",
	"language": "de"
},
{
	"summary": "I bought this product last month. I haven't tested it until now. I will do that during the next few weeks.",
	"sentiment": "neutral",
	"language": "en"
},
{
	"summary": "Ich weiß nicht was ich davon halten soll. Das Bild ist nicht scharf und die Farben sind blass. Ich würde es eher nicht empfehlen!",
	"sentiment": "negativ",
	"language": "de"
}
```


OutputParserException: Got invalid JSON object. Error: Extra data: line 5 column 2 (char 197)